In [1]:
!nvidia-smi

Fri Feb  9 13:06:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# Pip install YoloV8
!pip install ultralytics

# Supervision
!pip install supervision

# Roboflow
!pip install roboflow

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/64/98/e06ee1c8abb2be94743212cabf831885540a025c09a12c7f5016a6ca55b9/ultralytics-8.1.11-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 717.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.5/709.5 kB 5.8 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for supervision from https://files.pythonhosted.org/packages/53/f7/490cf47d2141f5cc8eeb944b90108cdea5ccfcd100fd6f669bc86cfb6278/supervision-0.18.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 937.0 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for roboflow from https://files.pythonhosted.org/packages/ae/bb/d95a8fe5470655a0f3051044040c2096f85658a327afecb42f715e4d1090/roboflow-1.1.19-py3-none-any.whl.metadata
  Obtaining dependency information for certifi==2023.7.22 from https://files.pythonhosted.org/packages/4c/dd/2234e

In [3]:
import ultralytics
ultralytics.checks()

import supervision as sv
print("supervision.__version__:", sv.__version__)

from ultralytics import YOLO

Ultralytics YOLOv8.1.11 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5432.2/8062.4 GB disk)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


supervision.__version__: 0.18.0


In [4]:
# Importing General libraries
import cv2
import pandas as pd
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
from roboflow import Roboflow

**Check different [Yolov8 models](http://docs.ultralytics.com/models/yolov8/#supported-tasks-and-modes)**

In [5]:
# Downloading dataset from Roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("human-face-detection").project("face-detection-0omif")
dataset = project.version(6).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.11, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Face-detection-6 in yolov8:: 100%|██████████| 4104/4104 [00:00<00:00, 4583.25it/s]


In [ ]:
### TO open the data.yaml file and Edit the dataset PATH

# %load /kaggle/working/Face-detection-6/data.yaml

# %%writefile /kaggle/working/Face-detection-4/data.yaml

In [ ]:
# %load /kaggle/working/Face-detection-6/data.yaml
names:
- human_face
nc: 1
roboflow:
  license: CC BY 4.0
  project: face-detection-0omif
  url: https://universe.roboflow.com/human-face-detection/face-detection-0omif/dataset/6
  version: 6
  workspace: human-face-detection
test: ../test/images
train: Face-detection-6/train/images
val: Face-detection-6/valid/images


In [6]:
%%writefile /kaggle/working/Face-detection-6/data.yaml
names:
- human_face
nc: 1
roboflow:
  license: CC BY 4.0
  project: face-detection-0omif
  url: https://universe.roboflow.com/human-face-detection/face-detection-0omif/dataset/6
  version: 6
  workspace: human-face-detection
test: /kaggle/working/Face-detection-6/test/images
train: /kaggle/working/Face-detection-6/train/images
val: /kaggle/working/Face-detection-6/valid/images


Overwriting /kaggle/working/Face-detection-6/data.yaml


In [7]:
# Load a model
# model = YOLO('yolov8n.yaml')  # build a new model from YAML
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
# model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

100%|██████████| 6.23M/6.23M [00:00<00:00, 69.7MB/s]


In [8]:
# Train the model with 2 GPUs
results = model.train(
    data = '/kaggle/working/Face-detection-6/data.yaml',
    epochs = 100,
    patience = 10,
    imgsz = 640,
    device = [0, 1],
)    

Ultralytics YOLOv8.1.11 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/Face-detection-6/data.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

100%|██████████| 755k/755k [00:00<00:00, 13.5MB/s]
2024-02-09 13:08:01,382	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-02-09 13:08:02,027	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


TensorBoard: Start with 'tensorboard --logdir runs/detect/train', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=1
Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


...

                   all        100        100      0.999          1      0.995      0.967

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     62/100      1.23G     0.2091     0.2041     0.8541         26        640: 100%|██████████| 121/121 [00:40<00:00,  2.99it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:00<00:00,  8.82it/s]


                   all        100        100      0.998          1      0.995      0.975
Stopping training early as no improvement observed in last 10 epochs. Best results observed at epoch 52, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

62 epochs completed in 0.736 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.11 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  3.47it/s]


                   all        100        100      0.999          1      0.995       0.98
Speed: 2.8ms preprocess, 2.7ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/train


# Saving the model to Google colab/google drive

In [9]:
!cp "/kaggle/working/runs/detect/train/weights/best.pt" "/kaggle/working/"

**Use the link to download the file in colab or locally**

In [10]:
from IPython.display import FileLink
FileLink(r'best.pt')

/kaggle/working/best.pt

In [ ]:
plt.imshow(cv2.imread('/kaggle/working/runs/detect/train/results.png'))
plt.show()